In [1]:
import matplotlib.pyplot as plt
import torch
from bin.model import BrainMimic
import networkx as nx
import nibabel as nib
import pandas as pd
import json
import os

220614-16:12:03,667 nipype.utils WARNING:
	 A newer version (1.8.1) of nipy/nipype is available. You are using 1.6.1
Intel Hardware Acceleration is not enabled. 


In [2]:
structure_graph_path = '/Users/loggiasr/Projects/fmri/monkey_fmri/MTurk1_frozen_20220527/misc_testing_files/fully_connected_simple.gml'  # path to shape color graph
data = json.load(open(structure_graph_path, 'r'))
gt = nx.readwrite.node_link_graph(data)

In [3]:
# prune gt
for s, t, data in list(gt.edges(data=True)):
    if abs(data['weight']) < .5:
        gt.remove_edge(s, t)

In [4]:
roi_atlas = nib.load('/Users/loggiasr/Projects/fmri/monkey_fmri/MTurk1_frozen_20220527/D99_v2.0_dist/simplified_atlas.nii')

In [5]:
start_node = None
for n, data in gt.nodes(data=True):
    if data['roi_name'] == 'V1':
        start_node = n
        break
echo = BrainMimic(gt, start_node, units_per_voxel=1, stimuli_shape=(1, 3, 64, 64), start_lr=.001)
# echo.load_network_state('/Users/loggiasr/Projects/fmri/monkey_fmri/MTurk1/misc_testing_files/brain_mimic_epoch_24')

added computational node 0 mPFC with size (1, 598, 1, 1)
added computational node 1 oPFC with size (1, 302, 2, 2)
added computational node 2 vlPFC with size (1, 109, 4, 4)
added computational node 3 ant-IT with size (1, 1325, 1, 1)
added computational node 4 STS with size (1, 745, 2, 2)
added computational node 5 hippocampus with size (1, 382, 1, 1)
added computational node 6 TE with size (1, 1095, 1, 1)
added computational node 7 V3 with size (1, 5, 16, 16)
added computational node 8 V3a with size (1, 2, 16, 16)
added computational node 9 V4 with size (1, 10, 16, 16)
added computational node 10 MT with size (1, 4, 16, 16)
added computational node 11 V2 with size (1, 5, 32, 32)
added computational node 12 V1 with size (1, 2, 64, 64)
added computational node 13 putamen with size (1, 1335, 1, 1)
added computational node 14 globbus-pallidus-internal with size (1, 156, 1, 1)
added computational node 15 globbus-pallidus-external with size (1, 352, 1, 1)
added computational node 16 claustrum

In [6]:
# load paradigm data
from dataloaders import dyloc_data_loader, shape_color_dataloader
stimuli = [shape_color_dataloader.ShapeColorBasicData("/Users/loggiasr/Projects/fmri/monkey_fmri/MTurk1/stimuli/small_stimuli", exp_image_size=(64, 64)),
           dyloc_data_loader.DylocDataloader("/Users/loggiasr/Projects/fmri/monkey_fmri/MTurk1/stimuli/dyloc_downsampled", exp_image_size=(64, 64), stim_frames=10)]


In [ ]:
# stimuli provided are looped in batch to match stimulus frame number
echo.fit_rdms(stimuli, epochs=100, stimulus_frames=30, verbose=True, snapshot_out='/Users/loggiasr/Projects/fmri/monkey_fmri/MTurk1/misc_testing_files',
              start_lr=.0001, final_lr=.00000001, prune_start=.02, prune_stop=.01)


********************
PRESENTING paradigm shape_color_basic
********************

PRESENTING cond achromatic_shape frame 0

PRESENTING cond achromatic_shape frame 1

PRESENTING cond achromatic_shape frame 2

PRESENTING cond achromatic_shape frame 3

PRESENTING cond achromatic_shape frame 4

PRESENTING cond achromatic_shape frame 5

PRESENTING cond achromatic_shape frame 6

PRESENTING cond achromatic_shape frame 7

PRESENTING cond achromatic_shape frame 8

PRESENTING cond achromatic_shape frame 9

PRESENTING cond achromatic_shape frame 10

PRESENTING cond achromatic_shape frame 11

PRESENTING cond achromatic_shape frame 12

PRESENTING cond achromatic_shape frame 13

PRESENTING cond achromatic_shape frame 14

PRESENTING cond achromatic_shape frame 15

PRESENTING cond achromatic_shape frame 16

PRESENTING cond achromatic_shape frame 17

PRESENTING cond achromatic_shape frame 18

PRESENTING cond achromatic_shape frame 19

PRESENTING cond achromatic_shape frame 20

PRESENTING cond achromati

In [ ]:
nx.draw_networkx(echo.brain, with_labels=True)

In [ ]:
import numpy as np
weights = []

comp = sorted(nx.connected_components(echo.brain.to_undirected()), key=len, reverse=True)
ai_graph = echo.brain.subgraph(comp[0]).copy()

edges = list(ai_graph.edges(data=True))

for node, data in list(ai_graph.nodes(data=True)):
    name = data['roi_name']
    ai_graph.remove_node(node)
    ai_graph.add_node(int(node), roi_name=str(name))

for u, v, data in edges:
    if 'sequence' in data:
        weight = float(torch.mean(data['sequence'][0].weight.detach().flatten()).item())
        weights.append(weight)
        ai_graph.add_edge(u, v, weight=weight)
threshold = 2 * np.std(np.array(weights))

In [ ]:
for s, t, data in list(ai_graph.edges(data=True)):
    if np.abs(data['weight']) < threshold:
        ai_graph.remove_edge(s, t)

In [ ]:
# rep graph
import graspologic as gr
pos_graph = ai_graph.copy()
for s, t, data in pos_graph.edges(data=True):
    pos_graph.edges[(s, t)]['weight'] = float(np.abs(data['weight']))
undirected_connectome, layout = gr.layouts.layout_tsne(pos_graph,
                                                       perplexity=10,
                                                       n_iter=1000)

In [ ]:
from graspologic.layouts.classes import NodePosition
for i in range(len(layout)):
    new_data = gr.layouts.classes.NodePosition(node_id=int(layout[i].node_id),
                                               x=layout[i].x,
                                               y=layout[i].y,
                                               size=layout[i].size,
                                               community=layout[i].community)
    layout[i] = new_data
cmap = plt.get_cmap('viridis')

raw_edge_weights = np.array([edge[2]['weight'] for edge in undirected_connectome.edges(data=True)])
edge_weights = raw_edge_weights - min(raw_edge_weights)
edge_weights = edge_weights / max(edge_weights)

edge_colors = np.array([cmap(val) for val in edge_weights])

edge_widths = np.abs(raw_edge_weights) - np.min(raw_edge_weights)
edge_widths = edge_widths / max(edge_widths)
edge_widths = (1. * edge_widths) + .25

node_colors = ['#808080'] * len(undirected_connectome.nodes())
labels = {}
for n in undirected_connectome.nodes:
    if 'roi_name' in undirected_connectome.nodes[n]:
        labels[n] = undirected_connectome.nodes[n]['roi_name']
    else:
        labels[n] = 'unnamed'

In [ ]:
# add edge colors
def rgb_to_hex(r, g, b):
  return ('{:X}{:X}{:X}').format(int(r*255), int(g*255), int(b*255))


for i, n in enumerate(undirected_connectome.nodes()):
    ai_graph.nodes[n]['graphics'] = {'x': float(layout[i].x),
                                  'y': float(layout[i].y),
                                  'w': float(layout[i].size),
                                  'h': float(layout[i].size)}

In [ ]:
drawing = gr.layouts.render._draw_graph(undirected_connectome, layout, node_colors, 1.,
                                        edge_line_width=edge_widths,
                                        edge_alpha=1.,
                                        edge_colors=edge_colors,
                                        labels=labels,
                                        figure_width=25,
                                        figure_height=25)

In [ ]:
data = nx.readwrite.write_gml(ai_graph, '/Users/loggiasr/Projects/fmri/monkey_fmri/MTurk1/misc_testing_files/cynetwork_out__mk1.gml')
